# Data Analysis

## View some data

In [ ]:
from constants.data_constants import JORDAN_DATASET_FILEPATH, MAESTRO_DATASET_FILEPATH
from data.jordan_dataset import JordanDataset
from data.maestro_dataset import MaestroDataset

id_train_dataset = JordanDataset(
    data_dir=JORDAN_DATASET_FILEPATH,
    split="train",
    name="id_train_dataset"
)
id_test_dataset = JordanDataset(
    data_dir=JORDAN_DATASET_FILEPATH,
    split="validation",
    name="id_test_dataset"
)
ood_test_dataset = MaestroDataset(
    data_dir=MAESTRO_DATASET_FILEPATH,
    split="test",
    name="maestro_test_dataset"
)

/data/scratch/joeltjy1/conda_envs/ood-detection/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading train split from /scratch/joel/jordan_dataset...
Loaded 4060 samples from train split
Sample keys: ['input_ids', 'labels']
Skipped 490 bad samples
Loading validation split from /scratch/joel/jordan_dataset...
Loaded 84 samples from validation split
Sample keys: ['input_ids', 'labels']
Skipped 0 bad samples
path /scratch/joel/maestrodata/test.txt


In [2]:
import random
import pandas as pd
from utils.process_tokens import get_readable_events

events_by_dataset = {}
for name, dataset in [("id_train", id_train_dataset), ("id_test", id_test_dataset), ("ood_test", ood_test_dataset)]:
    random_idx = random.randint(0, len(dataset) - 1)
    input_ids = dataset[random_idx]['input_ids']
    if hasattr(input_ids, 'tolist'):
        input_ids = input_ids.tolist()
    events = get_readable_events(input_ids)
    events_by_dataset[name] = events

min_events = min(len(events) for events in events_by_dataset.values())
num_rows = min(min_events, 30)

display_keys = ["misc", "onset", "dur", "inst", "pitch", "antic"]
key_mapping = {
    "misc": "special_token",
    "onset": "onset",
    "dur": "duration",
    "inst": "instrument",
    "pitch": "pitch",
    "antic": "anticipated"
}

columns = pd.MultiIndex.from_product([events_by_dataset.keys(), display_keys], names=['dataset', 'key'])

table_data = []
for event_idx in range(num_rows):
    row = []
    for dataset_name in events_by_dataset.keys():
        events = events_by_dataset[dataset_name]
        event = events[event_idx]
        for display_key in display_keys:
            actual_key = key_mapping[display_key]
            value = event.get(actual_key, "")
            if hasattr(value, 'item'):
                value = value.item()
            elif hasattr(value, 'tolist'):
                value = value.tolist()
            row.append(value)
    table_data.append(row)

df = pd.DataFrame(table_data, columns=columns)
df

dataset id_train                               id_test                         \
key         misc onset   dur inst pitch  antic    misc onset   dur inst pitch   
0             AR                                    AR                          
1                  0.0  0.61    0    A2  False           0.0  0.17    0    G2   
2                 4.47  0.34    0    G3  False          0.01  0.14    0    G4   
3                 4.66  0.29    0   A#3  False          0.09  0.11    0    F4   
4                 4.96  0.23    0    G3  False          0.11  0.13    0    F2   
5                 0.74  0.24    0    C5  False          0.21  0.13    0    C4   
6                 5.16   0.1    0    D4  False          0.22  0.14    0    C2   
7                 5.16  0.09    0   A#3  False           0.3  0.13    0    G4   
8                 5.51  0.19    0    E4  False          0.31   0.1    0    G2   
9                 5.52  0.51    0    G3  False          0.39  0.12    0    F2   
10                5.52  0.54    0    G2  False          0.42  0.12    0    F4   
11                1.41  0.13    0    D5  False           0.5  0.13    0   D#2   
12                6.21  0.37    0    G4  False          0.51   0.1    0   D#4   
13                6.21  0.43    0   A#3  False          0.87  0.06    0    G3   
14                6.22  0.56    0    D4  False          0.89  0.12    0    G1   
15                6.23  0.21    0    G3  False          0.98  0.11    0    C4   
16                6.23  1.74    0    G2  False          0.99  0.14    0    C2   
17                6.24  0.58    0    E3  False          1.09  0.08    0    D2   
18                 1.6  0.29    0    E5  False           1.1  0.09    0    D4   
19                6.57  0.06    0    G3  False           1.2  0.11    0    G4   
20                1.61  0.18    0    E4  False          1.21  0.08    0    G2   
21                 1.8  0.11    0    C4  False           1.3   0.1    0    D2   
22                6.75   0.1    0    A3  False          1.31  0.08    0    D4   
23                1.92  0.06    0    E4  False          1.41  0.08    0    C2   
24                2.11  0.74    0    F5  False          1.41  0.04    0    C4   
25                7.08   0.6    0    E3  False          1.51  0.23    0    F4   
26                2.11  0.49    0    A4  False          1.52  0.23    0    F2   
27                2.11  0.36    0    D5  False          1.73  0.13    0    D2   
28                2.12  1.23    0    D3  False          1.75  0.12    0    D4   
29          REST  3.12   0.0             False          1.85  0.08    0    C2   

dataset        ood_test                                
key      antic     misc onset   dur inst pitch  antic  
0                    AR                                
1        False            0.0  0.29    0    D4  False  
2        False            0.0  0.32    0    B3  False  
3        False           0.13  0.05    0   G#2  False  
4        False           0.27  0.06    0    A2  False  
5        False           0.27   0.6    0   C#4  False  
6        False           0.41  0.04    0    E2  False  
7        False           0.55  0.06    0    A2  False  
8        False           0.56  0.06    0    A3  False  
9        False           0.68  0.05    0    B2  False  
10       False            0.7  0.09    0   G#3  False  
11       False           0.82  0.05    0   C#3  False  
12       False           0.84   0.6    0    A3  False  
13       False           0.97  0.05    0    D3  False  
14       False           1.11  0.05    0    E3  False  
15       False           1.11  0.08    0   C#4  False  
16       False           1.24  0.06    0   C#3  False  
17       False           1.38  0.07    0    D3  False  
18       False            1.4  0.51    0   F#4  False  
19       False           1.53  0.06    0    E3  False  
20       False           1.67  0.07    0    D3  False  
21       False           1.68  0.07    0    B3  False  
22       False            1.8  0.05    0   C#3  False  
23     

## Running all multivariate tests

In [3]:
from data_analysis.mardia import mardia
from data_analysis.royston import royston
from data_analysis.hz import hz
from extract_layers.pooling_functions import pool_mean_std

num_layers = 24

tests = [mardia, royston, hz]

pooling_function = pool_mean_std

## Extract layers

### Sanity check

On an actually multivariate normal distribution, all tests should return a p-value of close to 1.

In [4]:
import numpy as np

X = np.random.randn(4000, 100)
metrics = {}
for test in tests:
    metrics = metrics | test(X)

print(metrics)




  Subsampling to 1000 samples and 100 features for testing
p_value 1.0
{'skewness': np.float64(172492.456), 'kurtosis': np.float64(-1.052), 'skewness_p': 0.088, 'kurtosis_p': 0.293, 'royston_p': 0.8, 'hz_p_value': 1.0}


In [ ]:
import os
import numpy as np
import pandas as pd

from constants.data_constants import SCRATCH_FILEPATH
from tqdm import tqdm


metrics = []

for layer_idx in tqdm(range(num_layers+1), desc="Processing layers"):
    layer_dir = os.path.join(SCRATCH_FILEPATH, pooling_function.__name__, f"layer_{layer_idx}.npy")
    layer_data = np.load(layer_dir)
    print("Processing layer", layer_idx)
    print("Layer data shape:", layer_data.shape)
    layer_metrics = {}
    for test in tests:
        print("Running test", test.__name__)
        layer_metrics = layer_metrics | test(layer_data)
    metrics.append(layer_metrics)

all_metrics = pd.DataFrame(metrics)


Processing layers:   0%|          | 0/25 [00:00<?, ?it/s]

Processing layer 0
Layer data shape: (4060, 2048)
Running test mardia
  Subsampling to 4060 samples and 100 features for testing
Running test royston
  Subsampling to 1000 samples and 100 features for testing
Running test hz
  Subsampling to 4060 samples and 100 features for testing


Processing layers:   4%|▍         | 1/25 [00:02<01:02,  2.61s/it]

p_value <0.001
Processing layer 1
Layer data shape: (4060, 2048)
Running test mardia
  Subsampling to 4060 samples and 100 features for testing
Running test royston
  Subsampling to 1000 samples and 100 features for testing
Running test hz
  Subsampling to 4060 samples and 100 features for testing


Processing layers:   8%|▊         | 2/25 [00:05<00:59,  2.58s/it]

...

p_value <0.001
Processing layer 23
Layer data shape: (4060, 2048)
Running test mardia
  Subsampling to 4060 samples and 100 features for testing
Running test royston
  Subsampling to 1000 samples and 100 features for testing
Running test hz
  Subsampling to 4060 samples and 100 features for testing


Processing layers:  96%|█████████▌| 24/25 [01:01<00:02,  2.60s/it]

p_value <0.001
Processing layer 24
Layer data shape: (4060, 2048)
Running test mardia
  Subsampling to 4060 samples and 100 features for testing
Running test royston
  Subsampling to 1000 samples and 100 features for testing
Running test hz
  Subsampling to 4060 samples and 100 features for testing


Processing layers: 100%|██████████| 25/25 [01:03<00:00,  2.55s/it]

p_value <0.001


In [6]:
all_metrics

,skewness,kurtosis,skewness_p,kurtosis_p,royston_p,hz_p_value
0,794505.571,188.220,0.001,0.001,0.001,0.001
1,820569.354,193.460,0.001,0.001,0.001,0.001
2,967104.005,175.458,0.001,0.001,0.001,0.001
3,1042388.043,153.681,0.001,0.001,0.001,0.001
4,1099449.310,162.002,0.001,0.001,0.001,0.001
5,1012898.224,160.416,0.001,0.001,0.001,0.001
6,1094736.248,162.257,0.001,0.001,0.001,0.001
7,1047037.065,145.477,0.001,0.001,0.001,0.001
8,981678.181,129.457,0.001,0.001,0.001,0.001
9,910557.318,101.975,0.001,0.001,0.001,0.001


In [7]:

## optionally save it
all_metrics.to_csv("all_metrics.csv", index=False)